#  RAG 성능평가 개요

### **학습 목표:** RAGAS를 사용한 RAG 성능 평가 프로세스를 이해한다

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import uuid

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

`(3) langfuase handler 설정`

In [3]:
#from langfuse.callback import CallbackHandler
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

# 정상 작동 확인
#langfuse_handler.auth_check() 

from langfuse import get_client
langfuse = get_client()
langfuse.auth_check()  # 인증 확인


True

---

## RAG 시스템 성능 평가

- **RAG 기술의 핵심**: 외부 지식 검색과 LLM 결합으로 응답 품질 향상

- **평가 기준**: LLM-as-judge 방식으로 사실성, 관련성, 충실도, 유용성 평가

- **체계적인 A/B 테스트**: 각 컴포넌트별 성능 비교 및 영향도 분석으로 최적 구성 도출

- **평가 방법론**: 오프라인(참조답변 기반), 온라인(실시간), 페어와이즈(비교) 평가 구분

<center>
<img src="ref/rag_evaluation.png" alt="rag" align="center" border="0"  width="1000" height=auto>
</center>
<center>
<img src="ref/rag_evaluation_target.png" alt="rag" align="center" border="0"  width="800" height=auto>
</center>


---

### **평가 대상(Evaluation Target)**

- **검색(Retrieval)** 단계: 
    1. 관련 문서와 쿼리 간의 연관성(Relevance)을 통해 검색된 문서가 쿼리의 정보 요구를 얼마나 잘 충족하는지 평가
    1. 관련 문서와 후보 문서 간의 정확성(Accuracy)을 통해 시스템이 적절한 문서를 식별하는 능력을 측정

- **생성(Generation)** 단계:
    1. 응답과 쿼리의 연관성(Relavance)
    1. 응답과 관련 문서 간의 충실도(Faithfulness)
    1. 응답과 샘플 응답 간의 정확성(Correctness)

- 추가 고려사항:
    - **핵심 성능 지표**: Latency(응답 속도), Diversity(검색 다양성), Noise Robustness(잡음 내구성)
    - **안전성 평가**: Negative Rejection(불충분 정보 거부), Counterfactual Robustness(오정보 식별)
    - **사용자 경험**: Readability(가독성), Toxicity(유해성), Perplexity(복잡성) 등 추가 고려


[출처] https://arxiv.org/abs/2405.07437

---

### **평가 데이터셋 구축(Evaluation Dataset)**

- **데이터셋 구성 방식**: LLM 기반 새로운 데이터셋 생성 (Synthetic Data)

- **맞춤형 데이터셋** 구축으로 RAG 시스템의 실용성 평가 강화


- [실습] : **Ragas** (https://docs.ragas.io/en/stable/) 활용

    - **RAG 시스템 평가**를 위한 오픈소스 프레임워크
    - **실용성**: 자동화된 평가 파이프라인 구축 가능
    - **주요 지표**: 

        1. **충실도(Faithfulness)**: 
            - 생성된 답변이 주어진 컨텍스트와 얼마나 일치하는지 평가
        
        2. **답변 관련성(Answer Relevancy)**:
            - 생성된 답변이 주어진 질문과 얼마나 관련이 있는지 평가
        
        3. **컨텍스트 정확도(Context Precision)**:
            - 검색된 컨텍스트들이 얼마나 적절하게 순위가 매겨졌는지 평가
            - 0~1 사이의 값으로, 높을수록 좋음


`(1) LangChain 문서 준비`

In [4]:
from langchain_community.document_loaders import TextLoader

# 데이터 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

korean_txt_files = glob(os.path.join('data', '*_KR.md')) 
korean_data = load_text_files(korean_txt_files)

print('Korean data:')
pprint(korean_data)

Korean data:
[Document(metadata={'source': 'data\\리비안_KR.md'}, page_content='Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.\n\n**주요 정보:**\n\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러\n- **총 자본 (2023):** 91억 4천만 미국 달러\n- **직원 수 (2023년 12월):** 16,790명\n- **웹사이트:** rivian.com\n\n**개요**\n\nRivian은 "스케이트보드" 플랫폼(R1T 및 R1S 모델)을 기반으로 한 전기 스포츠 유틸리티 차량(SUV), 픽업 트럭 및 전기 배달 밴(Rivian EDV)을 생산합니다. R1T 배송은 2021년 말에 시작되었습니다. 회사는 2022년에 미국에서 충전 네트워크를 시작하여 2024년에 다른 차량에도 개방했습니다. 생산 공장은 일리노이 주 노멀에 있으며, 다른 시설은 미국, 캐나다, 영국 및 세르비아의 여러 주에 있습니다.\n\n**역사**\n\n**초창기 (2009–15):**\n\n- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.\n- 2011년 Rivian Automotive로 사명 변경.\n- 처음에는 스포츠카 프로토타입(R1)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문장을 구분하여 분할 - 정규표현식 사용 (문장 구분자: 마침표, 느낌표, 물음표 다음에 공백이 오는 경우)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base",    # TikToken 인코더 이름
    separators=['\n\n', '\n', r'(?<=[.!?])\s+'],   # 구분자
    chunk_size=300,
    chunk_overlap=0,
    is_separator_regex=True,      # 구분자가 정규식인지 여부
    keep_separator=True,          # 구분자 유지 여부
)

korean_docs = text_splitter.split_documents(korean_data)

print("한국어 문서 수:", len(korean_docs))
print("-"*100)
print(korean_docs[0].metadata)
pprint(korean_docs[0].page_content)
print("-"*100)
print(korean_docs[1].metadata)
pprint(korean_docs[1].page_content)

한국어 문서 수: 39
----------------------------------------------------------------------------------------------------
{'source': 'data\\리비안_KR.md'}
('Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 '
 '회사입니다.\n'
 '\n'
 '**주요 정보:**')
----------------------------------------------------------------------------------------------------
{'source': 'data\\리비안_KR.md'}
('- **회사 유형:** 상장\n'
 '- **거래소:** NASDAQ: RIVN\n'
 '- **설립:** 2009년 6월, 플로리다 주 록ledge\n'
 '- **설립자:** R. J. 스캐린지\n'
 '- **본사:** 미국 캘리포니아 주 어바인\n'
 '- **서비스 지역:** 북미\n'
 '- **주요 인물:** R. J. 스캐린지 (CEO)\n'
 '- **제품:** 전기 자동차, 배터리\n'
 '- **생산량 (2023):** 57,232대\n'
 '- **서비스:** 전기 자동차 충전, 자동차 보험\n'
 '- **수익 (2023):** 44억 3천만 미국 달러\n'
 '- **순이익 (2023):** -54억 미국 달러\n'
 '- **총 자산 (2023):** 168억 미국 달러')


In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성하기
vector_store = Chroma.from_documents(
    documents=korean_docs,
    embedding=embedding_model,    
    collection_name="db_korean_cosine", 
    persist_directory="./chroma_db",
    collection_metadata = {'hnsw:space': 'cosine'}, # l2, ip, cosine 중에서 선택 
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 39


In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델을 로드
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 로드
vector_store = Chroma(
    embedding_function=embedding_model,    
    collection_name="db_korean_cosine", 
    persist_directory="./chroma_db",
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 39


`(2) LLM 설정`

In [8]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-mini"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

`(3) Test Data 생성`

In [9]:
from ragas.testset.persona import Persona

# 페르소나 정의 (다양한 관점에서 질문 생성)
personas = [
    Persona(
        name="graduate_researcher",  # 박사과정 연구원: 심도 있는 분석적 질문
        role_description="미국 전기차 시장을 연구하는 한국인 박사과정 연구원으로, 전기차 정책과 시장 동향에 대해 깊이 있는 분석을 하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="masters_student",    # 석사과정 학생: 개념 이해를 위한 질문
        role_description="전기차 산업을 공부하는 한국인 석사과정 학생으로, 미국 전기차 시장의 기초적인 개념과 트렌드를 이해하려 노력하고 있습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="industry_analyst",   # 산업 분석가: 실무 중심적 질문
        role_description="한국 자동차 회사에서 미국 전기차 시장을 분석하는 주니어 연구원으로, 실무적인 시장 데이터와 경쟁사 동향에 관심이 많습니다. 한국어만을 사용합니다.",
    ),
    Persona(
        name="undergraduate_student",  # 학부생: 기초적인 학습 질문
        role_description="자동차 공학을 전공하는 한국인 학부생으로, 미국 전기차 기술과 시장에 대해 기본적인 지식을 습득하고자 합니다. 한국어만을 사용합니다.",
    )
]

In [10]:
from ragas.testset import TestsetGenerator

# TestsetGenerator 생성
generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings, persona_list=personas)

# 합성 데이터 생성
dataset = generator.generate_with_langchain_docs(korean_docs, testset_size=50) 

Applying SummaryExtractor:   0%|          | 0/34 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/39 [00:00<?, ?it/s]

Node 89c738d1-a1c3-4183-810b-6e087c19c55e does not have a summary. Skipping filtering.
Node 0d51e742-0ea0-471e-970a-3c9e044677cd does not have a summary. Skipping filtering.
Node b2cc243b-4716-4908-bf20-35c0d3bc0530 does not have a summary. Skipping filtering.
Node 8056517f-0029-4d95-9d74-eb753b0047db does not have a summary. Skipping filtering.
Node c01b08b6-59f9-4a50-9488-94043440d775 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/112 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/51 [00:00<?, ?it/s]

In [12]:
# 노트북에 이미 정의된 영어 문서 사용
english_txt_files = glob(os.path.join("data", "*_EN.md"))
english_data = load_text_files(english_txt_files)
english_docs = text_splitter.split_documents(english_data)

# 영어 문서로 데이터셋 생성 (안전한 테스트)
dataset_en = generator.generate_with_langchain_docs(
    english_docs,
    testset_size=10
)

print("영어 문서로 테스트 완료!")

Applying SummaryExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/19 [00:00<?, ?it/s]

Node 34206c8d-4da6-4c2f-a276-aa5b017228f1 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

영어 문서로 테스트 완료!


In [13]:
test_data = dataset.to_pandas()
test_data

,user_input,reference_contexts,reference,synthesizer_name
0,"Rivian Automotive, Inc.는 어떤 회사인가요?","[Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동...","Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차...",single_hop_specifc_query_synthesizer
1,리브니 회사 배터리 어떻게 중요해요 전기차 시장에서?,[- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- *...,"리브니 회사는 전기 자동차와 배터리를 주요 제품으로 생산하며, 북미 지역에서 서비스...",single_hop_specifc_query_synthesizer
2,2023년에 리비안의 총 자본과 직원 수는 각각 얼마인가요?,[- **총 자본 (2023):** 91억 4천만 미국 달러\n- **직원 수 (2...,"2023년에 리비안의 총 자본은 91억 4천만 미국 달러이며, 2023년 12월 기...",single_hop_specifc_query_synthesizer
3,2022년 Rivian 미국 충전 네트워크 시작에 대해 알려줘?,"[**개요**\n\nRivian은 ""스케이트보드"" 플랫폼(R1T 및 R1S 모델)을...","Rivian은 2022년에 미국에서 충전 네트워크를 시작했으며, 이 네트워크는 20...",single_hop_specifc_query_synthesizer
4,R1S가 Rivian의 전기차 라인업에서 어떤 역할을 하는지 설명해 주시겠습니까?,[- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.\n-...,"R1S는 Rivian이 2017년 12월에 공개한 첫 두 제품 중 하나로, SUV ...",single_hop_specifc_query_synthesizer
5,R1T 배송은 2021년 9월에 언제 시작됬나요?,[- R1T 배송은 2021년 9월에 시작되어 Rivian은 완전 전기 픽업을 소비...,"R1T 배송은 2021년 9월에 시작되었으며, 이로 인해 Rivian은 완전 전기 ...",single_hop_specifc_query_synthesizer
6,R3가 뭐에요?,"[- 2022년 10월, 느슨한 토크 볼트로 인해 13,000대의 차량을 자발적으로...",R3는 2024년 3월에 공개된 더 작은 가격대의 SUV 모델입니다.,single_hop_specifc_query_synthesizer
7,R1S what car is?,"[**Volkswagen과의 파트너십 (2024)**\n\n- 2024년 6월, V...",R1S는 Rivian의 첫 번째 플랫폼을 기반으로 한 스포츠 유틸리티 차량(SUV)...,single_hop_specifc_query_synthesizer
8,2023년 4월에 Rivian이 발표한 중요한 변화는 무엇인가요?,[**EV 충전**\n\nRivian은 미국과 캐나다 전역에 공공 충전소 네트워크를...,2023년 4월에 Rivian은 Rivian Adventure Network가 Ri...,single_hop_specifc_query_synthesizer
9,Could you please explain the specific roles an...,"[- **Irvine, California:** 차량 엔지니어링 및 설계에 중점을 ...","The Plymouth, Michigan facility focuses on veh...",single_hop_specifc_query_synthesizer


In [14]:
# CSV 저장
test_data.to_csv('./data/ragas_testset.csv', index=False)

### **[실습] 평가 데이터셋 합성**

- "data/*_EN.md" 파일들을 문서로 가져와서 청크 길이를 300 토큰 단위로 분할
- ragas 활용 테스트 데이터셋 5개 생성

In [15]:
from langchain_community.document_loaders import TextLoader

# "data/*_EN.md" 파일 로드
def load_text_files(txt_files):
    data = []

    for text_file in txt_files:
        loader = TextLoader(text_file, encoding='utf-8')
        data += loader.load()

    return data

english_txt_files = glob(os.path.join('data', '*_EN.md'))
english_data = load_text_files(english_txt_files)

print('English data:')
pprint(english_data)

English data:
[Document(metadata={'source': 'data\\Rivian_EN.md'}, page_content='Rivian Automotive, Inc. is an American electric vehicle manufacturer, automotive technology, and outdoor recreation company founded in 2009.\n\n**Key Facts:**\n\n- **Company Type:** Public\n- **Traded As:** NASDAQ: RIVN\n- **Founded:** June 2009, in Rockledge, Florida\n- **Founder:** R. J. Scaringe\n- **Headquarters:** Irvine, California, U.S.\n- **Area Served:** North America\n- **Key People:** R. J. Scaringe (CEO)\n- **Products:** Electric vehicles, batteries\n- **Production Output (2023):** 57,232 vehicles\n- **Services:** Electric vehicle charging, vehicle insurance\n- **Revenue (2023):** US$4.43 billion\n- **Net Income (2023):** US$−5.4 billion\n- **Total Assets (2023):** US$16.8 billion\n- **Total Equity (2023):** US$9.14 billion\n- **Number of Employees (December 2023):** 16,790\n- **Website:** rivian.com\n\n**Overview**\n\nRivian produces electric sport utility vehicles (SUVs), pickup trucks on a "

In [16]:
# English 문서들을 300 토큰 단위로 분할
english_docs = text_splitter.split_documents(english_data)

print("English 문서 수:", len(english_docs))
print("-"*100)
print("첫 번째 영어 문서 메타데이터:", english_docs[0].metadata)
print("첫 번째 영어 문서 내용:")
print(english_docs[0].page_content[:200] + "...")
print("-"*100)

# English 전용 페르소나 정의 (영어 질문 생성)
english_personas = [
    Persona(
        name="ev_researcher",
        role_description="An automotive industry researcher specializing in electric vehicle market analysis in the United States. Focuses on technical specifications and market trends. Uses English only.",
    ),
    Persona(
        name="business_analyst", 
        role_description="A business analyst studying the competitive landscape of electric vehicle companies. Interested in financial performance and strategic positioning. Uses English only.",
    ),
    Persona(
        name="technology_student",
        role_description="An engineering student learning about electric vehicle technology and innovations. Seeks to understand technical concepts and industry developments. Uses English only.",
    )
]

# English용 TestsetGenerator 생성
english_generator = TestsetGenerator(
    llm=generator_llm, 
    embedding_model=generator_embeddings, 
    persona_list=english_personas
)

# RAGAS 활용 테스트 데이터셋 5개 생성
english_testset = english_generator.generate_with_langchain_docs(
    english_docs, 
    testset_size=5
)

print("영어 테스트 데이터셋 생성 완료!")

# 결과 확인
english_test_data = english_testset.to_pandas()
print(f"생성된 테스트 데이터 수: {len(english_test_data)}")
print("\n생성된 테스트 데이터 샘플:")
print(english_test_data[['user_input', 'reference']].head())

# CSV 저장
english_test_data.to_csv('./data/testset_en.csv', index=False)
print("\n테스트 데이터가 'data/testset_en.csv'에 저장되었습니다.")

English 문서 수: 19
----------------------------------------------------------------------------------------------------
첫 번째 영어 문서 메타데이터: {'source': 'data\\Rivian_EN.md'}
첫 번째 영어 문서 내용:
Rivian Automotive, Inc. is an American electric vehicle manufacturer, automotive technology, and outdoor recreation company founded in 2009.

**Key Facts:**

- **Company Type:** Public
- **Traded As:*...
----------------------------------------------------------------------------------------------------


Applying SummaryExtractor:   0%|          | 0/18 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/19 [00:00<?, ?it/s]

Node b90beda8-7ab8-48de-bd57-f6bd6f77fea3 does not have a summary. Skipping filtering.


Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/56 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/6 [00:00<?, ?it/s]

영어 테스트 데이터셋 생성 완료!
생성된 테스트 데이터 수: 6

생성된 테스트 데이터 샘플:
                                          user_input  \
0                 Wher was Rivian founded in Florid?   
1  Who is R. J. Scaringe and what is his role in ...   
2  Can you explain what Cybercab is and how it co...   
3  When Tesla started shipping the Cybertruck in ...   
4  Can you explane how the deliveries of Tesla's ...   

                                           reference  
0  Rivian Automotive, Inc. was founded in June 20...  
1  R. J. Scaringe founded Rivian in 2009 original...  
2  Cybercab is an upcoming two-passenger battery-...  
3  Tesla began shipping the Cybertruck in Novembe...  
4  Tesla's Model X, a mid-size luxury crossover S...  

테스트 데이터가 'data/testset_en.csv'에 저장되었습니다.


---

### **평가 지표(Evaluation Metric)**

#### 1) **검색(Retrieval) 평가**  

- **Non-Rank Based Metrics**: Accuracy, Precision, Recall@k 등을 통해 관련성의 이진적 평가를 수행

- **Rank-Based Metrics**: MRR(Mean Reciprocal Rank), MAP(Mean Average Precision)를 통해 검색 결과의 순위를 고려한 평가를 수행

- **RAG 특화 지표**: 기존 검색 평가 방식의 한계를 보완하는 LLM-as-judge 방식 도입

- **포괄적 평가**: 정확도, 관련성, 다양성, 강건성을 통합적으로 측정

#### 2) **생성(Generation) 평가**

- **전통적 평가**: ROUGE(요약), BLEU(번역), BertScore(의미 유사도) 지표 활용

- **LLM 기반 평가**: 응집성, 관련성, 유창성을 종합적으로 판단하는 새로운 접근법 도입 (전통적인 참조 비교가 어려운 상황에서 유용)

- **다차원 평가**: 품질, 일관성, 사실성, 가독성, 사용자 만족도를 포괄적 측정

- **상세 프롬프트**와 **사용자 선호도** 기준으로 생성 텍스트 품질 평가

`(1) RAG 체인 - 평가 대상`

In [17]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 벡터 저장소 검색기 생성
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

# RAG 체인 
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 템플릿 생성
template = """Answer the question based only on the following context:

[Context]
{context}

[Question]
{query}

[Answer]
"""
prompt = ChatPromptTemplate.from_template(template)

qa_chain = prompt | llm | StrOutputParser()

def format_docs(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)


query = "Tesla는 언제 누가 만들었나?"

relevant_docs = retriever.invoke(query)
qa_chain.invoke({"context": format_docs(relevant_docs), "query": query})

'Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarpenning에 의해 설립되었습니다.'

`(2) 평가 수행을 위한 데이터셋 전처리`

In [19]:
# 데이터 로드
import pandas as pd
testset = pd.read_excel('data/testset.xlsx')

# 데이터 확인
testset.head()

,user_input,reference_contexts,reference,synthesizer_name
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","['Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",single_hop_specifc_query_synthesizer
1,Forbes Global 2000에서 테슬라 순위 뭐야?,['Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (...,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,single_hop_specifc_query_synthesizer
2,Tesla는 언제 누가 만들었나?,"['Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, M...","Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberha...",single_hop_specifc_query_synthesizer
3,Larry Page는 전기차 시장에서 어떤 역할을 했나요?,"[""Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전...",Larry Page는 전기차 시장에서 Tesla의 후속 자금 조달에 투자한 기업가 ...,single_hop_specifc_query_synthesizer
4,Toyota와 Tesla의 관계는 무엇인가요?,['Roadster 생산은 2008년에 시작되었습니다. 2009년 1월까지 Tesl...,"2010년 5월, Tesla는 캘리포니아 주 프리몬트의 NUMMI 공장을 Toyot...",single_hop_specifc_query_synthesizer


In [21]:
from ragas import EvaluationDataset

# 데이터셋 생성
dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in testset.itertuples():
    query = row.user_input   # 사용자 입력
    reference = row.reference  # 참조 답변
    relevant_docs = retriever.invoke(query)  # 검색된 문서
    response = qa_chain.invoke(      # RAG 체인 생성 답변 생성
        {
            "context": format_docs(relevant_docs),
            "query": query,
        },config={"callbacks": [langfuse_handler]})  
    
    dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
            "response": response,
            "reference": reference,
        }
    )

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [22]:
# 데이터프레임 변환하여 확인 
evaluation_dataset.to_pandas().head()

,user_input,retrieved_contexts,response,reference
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","[Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로서 전기 자동차(...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(..."
1,Forbes Global 2000에서 테슬라 순위 뭐야?,[Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (2...,Forbes Global 2000에서 테슬라는 69위에 랭크되었습니다.,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.
2,Tesla는 언제 누가 만들었나?,"[Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Mu...",Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarp...,"Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberha..."
3,Larry Page는 전기차 시장에서 어떤 역할을 했나요?,[Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략...,Larry Page는 Tesla의 후속 자금 조달에 투자자로 참여하여 전기차 시장에...,Larry Page는 전기차 시장에서 Tesla의 후속 자금 조달에 투자한 기업가 ...
4,Toyota와 Tesla의 관계는 무엇인가요?,[## 파트너\n\nTesla는 Panasonic과 파트너십을 맺고 있으며 리튬 공...,Toyota는 Tesla의 이전 파트너입니다.,"2010년 5월, Tesla는 캘리포니아 주 프리몬트의 NUMMI 공장을 Toyot..."


In [23]:
# 데이터 저장
evaluation_dataset.to_pandas().to_csv('data/evaluation_dataset.csv', index=False)

### **[실습] RAG 체인 구성 및 추론 실행**

- "data/*_EN.md" 문서에 대한 RAG 체인을 구성
- 이전에 구축한 테스트 데이터셋 5개에 대한 RAG 체인의 출력값을 생성

In [24]:
# 데이터 로드
testset = pd.read_csv('data/testset_en.csv')
testset.head(2)

,user_input,reference_contexts,reference,synthesizer_name
0,Wher was Rivian founded in Florid?,"['Rivian Automotive, Inc. is an American elect...","Rivian Automotive, Inc. was founded in June 20...",single_hop_specifc_query_synthesizer
1,Who is R. J. Scaringe and what is his role in ...,['Rivian produces electric sport utility vehic...,R. J. Scaringe founded Rivian in 2009 original...,single_hop_specifc_query_synthesizer


In [25]:
# English 문서용 벡터 저장소 생성
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI Embeddings 모델 로드 (영어용)
embedding_model_en = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma 벡터 저장소 생성 (영어 문서용)
vector_store_en = Chroma.from_documents(
    documents=english_docs,
    embedding=embedding_model_en,    
    collection_name="db_english_cosine", 
    persist_directory="./chroma_db_en",
    collection_metadata={'hnsw:space': 'cosine'},
)

print(f"English 벡터 저장소에 저장된 Document 개수: {len(vector_store_en.get()['ids'])}")

# English 문서용 RAG 체인 구성
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 벡터 저장소 검색기 생성 (영어용)
retriever_en = vector_store_en.as_retriever(search_kwargs={"k": 5})

# RAG 체인 구성 (영어용)
llm_en = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 영어용 템플릿 생성
template_en = """Answer the question based only on the following context:

[Context]
{context}

[Question]
{query}

[Answer]
"""
prompt_en = ChatPromptTemplate.from_template(template_en)

qa_chain_en = prompt_en | llm_en | StrOutputParser()

def format_docs_en(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)

# 테스트 데이터셋에 대한 RAG 체인 출력값 생성
print("\n=== RAG 체인 추론 실행 ===")

# 결과 저장을 위한 리스트
rag_results = []

# 각 테스트 데이터에 대해 RAG 체인 실행
for idx, row in testset.iterrows():
    query = row['user_input']
    reference = row['reference']
    
    print(f"\n[질문 {idx+1}] {query}")
    
    # 관련 문서 검색
    relevant_docs = retriever_en.invoke(query)
    
    # RAG 체인으로 답변 생성
    response = qa_chain_en.invoke({
        "context": format_docs_en(relevant_docs), 
        "query": query
    }, config={"callbacks": [langfuse_handler]})
    
    print(f"[RAG 답변] {response}")
    print(f"[참조 답변] {reference}")
    print("-" * 80)
    
    # 결과 저장
    rag_results.append({
        "query": query,
        "rag_response": response,
        "reference": reference,
        "retrieved_contexts": [doc.page_content for doc in relevant_docs]
    })

# 결과를 DataFrame으로 변환하여 저장
import pandas as pd
rag_results_df = pd.DataFrame(rag_results)
rag_results_df.to_csv('data/rag_results_en.csv', index=False)

print(f"\n=== 완료 ===")
print(f"총 {len(rag_results)}개 질문에 대한 RAG 체인 출력값을 생성했습니다.")
print("결과가 'data/rag_results_en.csv'에 저장되었습니다.")

English 벡터 저장소에 저장된 Document 개수: 19

=== RAG 체인 추론 실행 ===

[질문 1] Wher was Rivian founded in Florid?
[RAG 답변] Rivian was founded in Rockledge, Florida.
[참조 답변] Rivian Automotive, Inc. was founded in June 2009 in Rockledge, Florida.
--------------------------------------------------------------------------------

[질문 2] Who is R. J. Scaringe and what is his role in Rivian?
[RAG 답변] R. J. Scaringe is the founder of Rivian Automotive, Inc. and serves as the company's CEO.
[참조 답변] R. J. Scaringe founded Rivian in 2009 originally as Mainstream Motors. He led the company through its renaming to Rivian Automotive in 2011 and its shift from focusing on a sports car prototype to electric and autonomous vehicles.
--------------------------------------------------------------------------------

[질문 3] Can you explain what Cybercab is and how it compares to electric delivery vans like Rivian EDV in terms of their purpose and development?
[RAG 답변] The provided context does not mention or describe "

`(3) 평가 수행`

- **LLMContextRecall**
    - 검색 기반 응답 생성 시스템의 **컨텍스트 검색 성능**을 평가

- **Faithfulness** 
    - 검색된 컨텍스트에 대한 응답의 **충실도**를 측정하는 메트릭

- **FactualCorrectness**
    - 생성된 응답의 **사실 정확성**을 평가

In [26]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# LLM 래퍼 생성
evaluator_llm = LangchainLLMWrapper(llm)

# 평가
result = evaluate(
    dataset=evaluation_dataset,   # 평가 데이터셋
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],   # 평가 메트릭
    llm=evaluator_llm,   # LLM 래퍼
    callbacks=[langfuse_handler],   # 콜백
)

result

Evaluating:   0%|          | 0/147 [00:00<?, ?it/s]

{'context_recall': 0.8633, 'faithfulness': 0.8941, 'factual_correctness(mode=f1)': 0.6329}

In [27]:
# 결과를 데이터프레임으로 변환 
result.to_pandas()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,"Tesla, Inc.는 미국에서 어떤 역할을 하고 있으며, 이 회사의 주요 제품과 ...","[Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사입니다. 이 회사...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로서 전기 자동차(...","Tesla, Inc.는 미국의 다국적 자동차 및 청정 에너지 회사로, 전기 자동차(...",1.000000,0.769231,0.57
1,Forbes Global 2000에서 테슬라 순위 뭐야?,[Tesla의 차량 생산은 2008년 Roadster로 시작하여 Model S (2...,Forbes Global 2000에서 테슬라는 69위에 랭크되었습니다.,테슬라는 Forbes Global 2000에서 69위에 랭크되었습니다.,1.000000,1.000000,1.00
2,Tesla는 언제 누가 만들었나?,"[Tesla는 내부 고발자 보복, 근로자 권리 침해, 안전 결함, 홍보 부족, Mu...",Tesla는 2003년 7월 1일에 Martin Eberhard와 Marc Tarp...,"Tesla Motors, Inc.는 2003년 7월 1일에 Martin Eberha...",1.000000,1.000000,0.67
3,Larry Page는 전기차 시장에서 어떤 역할을 했나요?,[Elon Musk는 주류 차량으로 확장하기 전에 프리미엄 스포츠카로 시작하는 전략...,Larry Page는 Tesla의 후속 자금 조달에 투자자로 참여하여 전기차 시장에...,Larry Page는 전기차 시장에서 Tesla의 후속 자금 조달에 투자한 기업가 ...,1.000000,0.500000,0.80
4,Toyota와 Tesla의 관계는 무엇인가요?,[## 파트너\n\nTesla는 Panasonic과 파트너십을 맺고 있으며 리튬 공...,Toyota는 Tesla의 이전 파트너입니다.,"2010년 5월, Tesla는 캘리포니아 주 프리몬트의 NUMMI 공장을 Toyot...",0.000000,1.000000,0.50
5,Tesla의 Model 3에 대한 정보를 어디서 찾을 수 있나요?,[Tesla는 2016년 11월 SolarCity를 26억 달러에 인수하여 Tesl...,Tesla의 Model 3에 대한 정보는 다음 두 부분에서 찾을 수 있습니다:\n\...,제공된 문맥에는 Tesla의 Model 3에 대한 정보가 포함되어 있지 않습니다.,0.000000,1.000000,0.00
6,Tesla Energy 뭐야?,[Tesla Energy는 태양 에너지 생성 시스템과 배터리 에너지 저장 제품을 개...,Tesla Energy는 태양 에너지 생성 시스템과 배터리 에너지 저장 제품을 개발...,Tesla는 2016년 11월 SolarCity를 26억 달러에 인수하여 Tesla...,1.000000,1.000000,0.00
7,2020년 3월에 Tesla의 주요 사건은 무엇입니까?,[Tesla는 2016년 11월 SolarCity를 26억 달러에 인수하여 Tesl...,2020년 3월에 Tesla는 COVID-19 팬데믹으로 인해 프리몬트 공장을 폐쇄...,2020년 3월에 Tesla는 Model Y 중형 크로스오버 SUV의 배송을 시작했...,0.000000,1.000000,0.00
8,Megafactory가 Tesla의 사업 확장에 어떻게 기여했는지 설명해줄 수 있나요?,"[2023년 3월, Tesla는 잠재적인 관세 영향으로 인해 지연된 Gigafact...","Megafactory는 2022년에 설립된 Tesla의 새로운 시설로, 캘리포니아에...",Tesla는 2022년에 Megafactory를 통해 캘리포니아에서의 입지를 확장했...,1.000000,0.500000,0.36
9,Tesla가 Bitcoin에 투자하고 나중에 매각한 이유는 뭐였나요?,"[2021년 초, Tesla는 Bitcoin에 15억 달러를 투자하고 환경 문제로 ...",Tesla는 2021년 초에 Bitcoin에 15억 달러를 투자하고 잠시 결제 수단...,"Tesla는 2021년 초에 Bitcoin에 15억 달러를 투자했으나, 환경 문제로...",1.000000,0.800000,0.89


In [28]:
# 데이터프레임 저장
result.to_pandas().to_csv('data/evaluation_result.csv', index=False)

### **[실습] RAGAS 평가 수행**

- "data/*_EN.md" 문서에 대한 RAG 생성 답변을 평가 
- RAGAS 평가 메트릭 적용 (LLMContextRecall, Faithfulness, FactualCorrectness)

In [30]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from ragas import EvaluationDataset, evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# English 문서가 없으면 로드
if 'english_data' not in globals():
    english_txt_files = glob(os.path.join('data', '*_EN.md'))
    english_data = load_text_files(english_txt_files)

# 텍스트 분할 (기존 text_splitter 재사용 가능)
english_docs = text_splitter.split_documents(english_data)
print("English 문서 청크 수:", len(english_docs))

# 임베딩 및 Chroma 생성(영어 전용)
embedding_model_en = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store_en = Chroma.from_documents(
    documents=english_docs,
    embedding=embedding_model_en,
    collection_name="db_english_cosine",
    persist_directory="./chroma_db_en",
    collection_metadata={'hnsw:space': 'cosine'},
)

retriever_en = vector_store_en.as_retriever(search_kwargs={"k": 5})

# RAG 체인 구성 (간단한 prompt + LLM)
template = """Answer the question based only on the following context:

[Context]
{context}

[Question]
{query}

[Answer]
"""
prompt = ChatPromptTemplate.from_template(template)
llm_en = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
qa_chain_en = prompt | llm_en | StrOutputParser()

def format_docs(relevant_docs):
    return "\n".join(doc.page_content for doc in relevant_docs)

# 영어 테스트셋 로드(상위 5개만 사용)
testset_en = pd.read_csv('data/testset_en.csv')
testset_en = testset_en.head(5)

# EvaluationDataset 구성
dataset_list = []
for row in testset_en.itertuples(index=False):
    # 컬럼명이 다르면 적절히 바꾸세요 (예: user_input, reference)
    query = getattr(row, 'user_input', None) or getattr(row, 'query', None)
    reference = getattr(row, 'reference', None)
    if query is None:
        continue

    relevant_docs = retriever_en.invoke(query)
    response = qa_chain_en.invoke(
        {"context": format_docs(relevant_docs), "query": query},
        config={"callbacks": [langfuse_handler]},
    )

    dataset_list.append({
        "user_input": query,
        "retrieved_contexts": [rdoc.page_content for rdoc in relevant_docs],
        "response": response,
        "reference": reference,
    })

evaluation_dataset_en = EvaluationDataset.from_list(dataset_list)

# 평가 실행
evaluator_llm = LangchainLLMWrapper(llm_en)
result_en = evaluate(
    dataset=evaluation_dataset_en,
    metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness()],
    llm=evaluator_llm,
    callbacks=[langfuse_handler],
)

# 결과 확인 및 저장
print(result_en.to_pandas().head())
result_en.to_pandas().to_csv('data/evaluation_result_en.csv', index=False)


English 문서 청크 수: 19


Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

                                          user_input  \
0                 Wher was Rivian founded in Florid?   
1  Who is R. J. Scaringe and what is his role in ...   
2  Can you explain what Cybercab is and how it co...   
3  When Tesla started shipping the Cybertruck in ...   
4  Can you explane how the deliveries of Tesla's ...   

                                  retrieved_contexts  \
0  [Rivian Automotive, Inc. is an American electr...   
1  [Rivian Automotive, Inc. is an American electr...   
2  [- In June 2024, Volkswagen Group announced an...   
3  [In March 2023, Tesla announced plans for Giga...   
4  [- R1T shipments began in September 2021, maki...   

                                            response  \
0          Rivian was founded in Rockledge, Florida.   
1  R. J. Scaringe is the founder of Rivian Automo...   
2  The provided context does not mention or provi...   
3  When Tesla started shipping the Cybertruck in ...   
4  Based on the provided context, Rivian began